# Neural Nets from Scratch in Julia

## Lesson 18: Solving MNIST

* In this video we'll finally put everything together make a neural network model that can classify handwritten digits, using the MNIST dataset. The dataset used in this video can be downloaded [here.](https://www.kaggle.com/datasets/scolianni/mnistasjpg?resource=download)
* [Documentation site here](https://mikesaint-antoine.github.io/SimpleGrad.jl)
* [Github repo here](https://github.com/mikesaint-antoine/SimpleGrad.jl)

In [1]:
## code so far


struct Operation{FuncType, ArgTypes}
    op::FuncType
    args::ArgTypes
end


####################################################################################
###### Values

mutable struct Value{opType} <: Number
    data::Float64
    grad::Float64
    op::opType
end

# constructor -- Value(data, grad, op)
Value(x::Number) = Value(Float64(x), 0.0, nothing);


import Base.show
function show(io::IO, value::Value)
    print(io, "Value(",value.data,")")
end


import Base.==
function ==(a::Value, b::Value)
     return a===b
end


import Base.+
function +(a::Value, b::Value)

    out = a.data + b.data    
    result = Value(out, 0.0, Operation(+, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



backprop!(val::Value{Nothing}) = nothing


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(+), ArgTypes}
    
    # val = a + b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.grad
    val.op.args[2].grad += val.grad
    
end




function backward(a::Value)
    
    
    function build_topo(v::Value, visited=Value[], topo=Value[])
    
        if !(v in visited)
            
            push!(visited, v)
            
            if v.op != nothing
                for operand in v.op.args
                    
                    if operand isa Value
                        build_topo(operand, visited, topo)
                    end
                end 
            end
            
            push!(topo, v) 
            
            
        end
        return topo
    end
    
    
    
    topo = build_topo(a)
    
    a.grad = 1
    #da/da = 1
    
    for node in reverse(topo)
        backprop!(node)
    end
    
    
end


Base.promote_rule(::Type{<:Value}, ::Type{T}) where {T<:Number} = Value




import Base.*
function *(a::Value, b::Value)

    out = a.data * b.data    
    result = Value(out, 0.0, Operation(*, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(*), ArgTypes}
    
    # val = a * b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.op.args[2].data * val.grad    
    val.op.args[2].grad += val.op.args[1].data * val.grad
    
end



import Base.-

# negation
function -(a::Value)
    
    return a * -1
    
end

# subtraction
function -(a::Value, b::Value)
    
    return a + (-b)
    
    
end


import Base.inv
function inv(a::Value)
    
    out = 1.0 / a.data
    result = Value(out, 0.0, Operation(inv, (a,))) # Value(data, grad, op)
    return result # this should be a Value    
    
    
end


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(inv), ArgTypes}
    
    # val = inv(a)
    # update a.grad
    
    # a.grad -= (1.0 / a.data^2) * val.grad
    
    val.op.args[1].grad -= (1.0 / val.op.args[1].data^2) * val.grad
    
    
end


import Base./
function /(a::Value, b::Value)
     
    # a/b = a * b^(-1)
    
    return a * inv(b)
    
    
end


import Base.tanh
function tanh(a::Value)
    
    out = (exp(2 * a.data) - 1) / (exp(2 * a.data) + 1)
    result = Value(out, 0.0, Operation(tanh, (a,))) # Value(data, grad, op)
    return result # this should be a Value  
    
end




function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(tanh), ArgTypes}

    # val = tanh(a)
    # update a.grad
    
    val.op.args[1].grad += (1 - val.data^2) * val.grad
    

end

####################################################################################
###### Tensors

mutable struct Tensor{opType} <: AbstractArray{Float64, 2}
    data::Array{Float64,2}
    grad::Array{Float64,2}
    op::opType
end

# 2D constructor -- Tensor(data, grad, op)
Tensor(x::Array{Float64,2}) = Tensor(x, zeros(Float64,size(x)), nothing);

# 1D constructor
function Tensor(x::Array{Float64, 1}; column_vector::Bool=false)

    if column_vector
        # reshape x to column vector - size (N,1)
        data_2D = reshape(x, (length(x),1))

    else
        # DEFAULT - row vector - size (1,N)
        data_2D = reshape(x, (1, length(x)))

    end

    return Tensor(data_2D, zeros(Float64, size(data_2D)), nothing) # Tensor(data, grad, op)

end


import Base.show
function show(io::IO, tensor::Tensor)
    print(io, "Tensor(",tensor.data,")")
end

backprop!(tensor::Tensor{Nothing}) = nothing


import Base.==
function ==(a::Tensor, b::Tensor)
     return a===b
end


Base.size(x::Tensor) = size(x.data)

Base.getindex(x::Tensor, i...) = getindex(x.data, i...)

Base.setindex!(x::Tensor, v, i...) = setindex!(x.data, v, i...)


import Base.*
function *(a::Tensor, b::Tensor)

    out = a.data * b.data    
    result = Tensor(out, zeros(Float64, size(out)), Operation(*, (a,b))) # Tensor(data, grad, op)
    return result
 
end



function backprop!(tensor::Tensor{Operation{FunType, ArgTypes}}) where {FunType<:typeof(*), ArgTypes}
    
    # tensor = a * b
    # backprop!(tensor)
    # update a.grad, b.grad
    
    tensor.op.args[1].grad += tensor.grad * transpose(tensor.op.args[2].data)
    tensor.op.args[2].grad += transpose(tensor.op.args[1].data) * tensor.grad 
    
end


function backward(a::Tensor)
    
    
    function build_topo(v::Tensor, visited=Tensor[], topo=Tensor[])
    
        if !(v in visited)
            
            push!(visited, v)
            
            if v.op != nothing
                for operand in v.op.args
                    
                    if operand isa Tensor
                        build_topo(operand, visited, topo)
                    end
                end 
            end
            
            push!(topo, v) 
            
            
        end
        return topo
    end
    
    
    
    topo = build_topo(a)
    
    a.grad .= 1
    #da/da = 1
    
    for node in reverse(topo)
        backprop!(node)
    end
    
    
end


import Base.+
function +(a::Tensor, b::Tensor)

    # broadcasting happens automatically in case of row-vector
    out = a.data .+ b.data    

    result = Tensor(out, zeros(Float64, size(out)), Operation(+, (a,b))) # Tensor(data, grad, op)
    return result
 
end



function backprop!(tensor::Tensor{Operation{FunType, ArgTypes}}) where {FunType<:typeof(+), ArgTypes}
    
    # tensor = a + b
    # backprop!(tensor)
    # update a.grad, b.grad

    if size(tensor.grad) == size(tensor.op.args[1].data)
        tensor.op.args[1].grad += ones(size(tensor.op.args[1].data)) .* tensor.grad
    else
        # reverse broadcast
        tensor.op.args[1].grad += ones(size(tensor.op.args[1].grad)) .* sum(tensor.grad,dims=1)
    end

    
    if size(tensor.grad) == size(tensor.op.args[2].data)
        tensor.op.args[2].grad += ones(size(tensor.op.args[2].data)) .* tensor.grad
    else
        # reverse broadcast
        tensor.op.args[2].grad += ones(size(tensor.op.args[2].grad)) .* sum(tensor.grad,dims=1)
    end
    
    
end


function relu(a::Tensor)
    
    out = max.(0,a.data)
    result = Tensor(out, zeros(Float64, size(out)), Operation(relu, (a,))) # Tensor(data, grad, op)
    return result
    
end



function backprop!(tensor::Tensor{Operation{FunType, ArgTypes}}) where {FunType<:typeof(relu), ArgTypes}

    # tensor = relu(a)
    # update a.grad
    
    tensor.op.args[1].grad += (tensor.op.args[1].data .> 0) .* tensor.grad
    

end


function softmax_crossentropy(a::Tensor,y_true::Union{Array{Int,2},Array{Float64,2}}; grad::Bool=true)

    # softmax activation
    exp_values = exp.(a.data .- maximum(a.data, dims=2))
    probs = exp_values ./ sum(exp_values, dims=2)

    probs_clipped = clamp.(probs, 1e-7, 1 - 1e-7)
    # deal with 0s and 1s

    # basically just returns an array with the probability of the correct answer for each sample
    correct_confidences = sum(probs_clipped .* y_true, dims=2)   

    # negative log likelihood
    sample_losses = -log.(correct_confidences)

    # loss mean
    out = [sum(sample_losses) / length(sample_losses)]


    # ##
    if grad

        # calculate and update a.grad

        samples = size(probs,1)
        
        
        y_true_argmax = argmax(y_true, dims=2)
        
        a.grad = copy(probs)
        
        
        for samp_ind in 1:samples
            a.grad[samp_ind, y_true_argmax[samp_ind][2]] -=1
        end
        
        a.grad = a.grad ./ samples

    end


    # reshape out from (1,) to (1,1) 
    out = reshape(out, (1, 1))

    result = Tensor(out, zeros(Float64, size(out)), Operation(softmax_crossentropy, (a,))) # Tensor(data, grad, op)

    return result
    

end


# softmax_crossentropy backprop is empty because gradient is easier to calculate during forward pass
function backprop!(tensor::Tensor{Operation{FunType, ArgTypes}}) where {FunType<:typeof(softmax_crossentropy), ArgTypes}

end

backprop! (generic function with 10 methods)

In [2]:
## data link: https://www.kaggle.com/datasets/scolianni/mnistasjpg

# Pkg.add("Images")
using Images

In [12]:
# basic image processing

img_path = "archive/trainingSet/trainingSet/0/img_1.jpg"

img = load(img_path)

img_mat = Float64.(img)

img_flattened = reshape(img_mat, :)

# println(img_mat)
println(size(img_flattened))

(784,)


In [13]:
# reading in images

base_path = "archive/trainingSet/trainingSet/"

X = [] # image pixel data. (N,784)
y = [] # digit label. (N,)


for digit in 0:9
    folder_path = joinpath(base_path, string(digit))
    for file in readdir(folder_path)
        img_path = joinpath(folder_path, file)
        img = load(img_path)
        img_mat = Float64.(img)
        img_flattened = reshape(img_mat, :)
        push!(X, img_flattened)
        push!(y, digit)  
        
    end

end


X = hcat(X...)' # transposing to (N, 784)

println(size(X))
println(size(y))

(42000, 784)
(42000,)


In [15]:
using Random

n = size(X,1)

perm = shuffle(1:n)
X = X[perm, :]
y = y[perm];

In [16]:
train_size = Int(0.8 * size(X,1))

X_train = X[1:train_size, :]
y_train = y[1:train_size]

X_test = X[train_size+1:end, :]
y_test = y[train_size+1:end]

println(size(X_train))
println(size(y_train))

println(size(X_test))
println(size(y_test))


(33600, 784)
(33600,)
(8400, 784)
(8400,)


In [32]:
# initializing parameters
weights1 = Tensor(0.01 * rand(784, 128)) # taking in 784 input features, has 128 neurons in the layer
biases1 = Tensor(zeros(128))
weights2 = Tensor(0.01 * rand(128, 10)) # taking 128 inputs (from 128 neurons in the first layer), has 10 neurons in the layer
biases2 = Tensor(zeros(10))


# hyperparameters
batch_size = 100
num_classes = 10
epochs = 3
lr = 0.1;

In [34]:

global run = 1
for epoch in 1:epochs
    for i in 1:batch_size:size(X_train,1)
    
    
        # size of input matrix = (batch_size, 784)
        batch_X = Tensor(X_train[i:i+batch_size-1, :])
        batch_y = y_train[i:i+batch_size-1]
        
        
        
        ## convert batch_y to one-hot
        batch_y_one_hot = zeros(batch_size,num_classes)
        for batch_ind in 1:batch_size
            batch_y_one_hot[batch_ind,Int.(batch_y)[batch_ind]+1] = 1
        end
        
        
        ## zero grads
        weights1.grad .= 0
        weights2.grad .= 0
        biases1.grad .= 0
        biases2.grad .= 0
        
        
        # layer 1
        layer1_out = relu(batch_X * weights1 + biases1);
        
        # layer 2
        layer2_out = layer1_out * weights2 + biases2
        
        
        loss = softmax_crossentropy(layer2_out,batch_y_one_hot)
            
        
        backward(loss)
        
        
        # updating parameter values based on gradient
        weights1.data = weights1.data - weights1.grad .* lr
        biases1.data = biases1.data - biases1.grad .* lr
        weights2.data = weights2.data - weights2.grad .* lr
        biases2.data = biases2.data - biases2.grad .* lr;


        if run % 10 == 0
            println("Epoch: $epoch, run: $run, loss: $(round(loss.data[1], digits=3))")
        end
        
        global run += 1
    
    end
end

Epoch: 1, run: 10, loss: 2.302
Epoch: 1, run: 20, loss: 2.271
Epoch: 1, run: 30, loss: 2.267
Epoch: 1, run: 40, loss: 2.255
Epoch: 1, run: 50, loss: 2.152
Epoch: 1, run: 60, loss: 2.06
Epoch: 1, run: 70, loss: 1.898
Epoch: 1, run: 80, loss: 1.636
Epoch: 1, run: 90, loss: 1.567
Epoch: 1, run: 100, loss: 1.304
Epoch: 1, run: 110, loss: 1.077
Epoch: 1, run: 120, loss: 1.019
Epoch: 1, run: 130, loss: 0.849
Epoch: 1, run: 140, loss: 0.842
Epoch: 1, run: 150, loss: 0.866
Epoch: 1, run: 160, loss: 0.751
Epoch: 1, run: 170, loss: 0.672
Epoch: 1, run: 180, loss: 0.711
Epoch: 1, run: 190, loss: 0.799
Epoch: 1, run: 200, loss: 0.507
Epoch: 1, run: 210, loss: 0.549
Epoch: 1, run: 220, loss: 0.504
Epoch: 1, run: 230, loss: 0.511
Epoch: 1, run: 240, loss: 0.418
Epoch: 1, run: 250, loss: 0.556
Epoch: 1, run: 260, loss: 0.501
Epoch: 1, run: 270, loss: 0.567
Epoch: 1, run: 280, loss: 0.479
Epoch: 1, run: 290, loss: 0.467
Epoch: 1, run: 300, loss: 0.629
Epoch: 1, run: 310, loss: 0.481
Epoch: 1, run: 320

In [36]:
global correct = 0
global total = 0
for i in 1:length(y_test)
    X_in = X_test[i:i,:] ## need to keep this (1,784), not (784,)
    X_in = Tensor(X_in)
    y_true = y_test[i]

    layer1_out = relu(X_in * weights1 + biases1)
    layer2_out = layer1_out * weights2 + biases2


    pred_argmax = argmax(layer2_out.data, dims=2)[1][2]

    if pred_argmax-1 == y_true # -1 because digits start at 0
        global correct +=1
    end
    global total += 1

end

println("accuracy:")
println(correct/total)

accuracy:
0.9188095238095239


In [37]:
# user-friendly function

function guess_digit(img_path::String)
    
    # load image and conver to Tensor
    img = load(img_path)
    img_mat = Float64.(img)
    img_flattened = reshape(img_mat, :)
    img_tensor = Tensor(img_flattened)
    
    layer1_out = relu(img_tensor * weights1 + biases1)
    layer2_out = layer1_out * weights2 + biases2
    pred_argmax = argmax(layer2_out.data, dims=2)[1][2]
    prediction = pred_argmax-1 # because digits start at 0
    
    println("Guess: $prediction")
    
    
end

guess_digit (generic function with 1 method)

In [40]:
img_path = "archive/testSet/testSet/img_3.jpg"


guess_digit(img_path)

Guess: 9
